In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import warnings
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
warnings.filterwarnings('ignore')

In [3]:
hp_df=pd.read_csv('House Price Data.csv')
y=hp_df['SalePrice']

#Droping output variable from dataset

hp_df.drop(columns=['SalePrice','Id'],inplace=True)

nadf_info=pd.DataFrame({'col_name':hp_df.columns,'na_cnt':hp_df.isnull().sum(),'na_pct':(hp_df.isnull().sum()/hp_df.shape[0])*100})

cols_grt50_ls=list(nadf_info[nadf_info['na_pct']>50]['col_name'])

hp_df.drop(columns=cols_grt50_ls,inplace=True)

x_train,x_test,y_train,y_test=train_test_split(hp_df,y,random_state=49,test_size=0.25)

con_cols=[col for col in hp_df.columns if hp_df[col].dtype=='int64' or hp_df[col].dtype=='float64']

cat_cols=[col for col in hp_df.columns if hp_df[col].dtype=='object']

## filling missing values for contnous columns 
for col in con_cols:
    x_train[col].fillna(x_train[col].mean(),inplace=True)
    x_test[col].fillna(x_train[col].mean(),inplace=True)

## filling missing values for catgorical columns 
    
for col in cat_cols:
    x_train[col].fillna(x_train[col].mode()[0],inplace=True)
    x_test[col].fillna(x_train[col].mode()[0],inplace=True)


In [30]:
scalar=StandardScaler()

for col in con_cols:
    x_train[col]=scalar.fit_transform(np.array(x_train[col]).reshape(-1,1))
    x_test[col]=scalar.transform(np.array(x_test[col]).reshape(-1,1))


In [5]:
cat_encd_train= pd.get_dummies(x_train[cat_cols])
cat_encd_test=pd.get_dummies(x_test[cat_cols])

In [6]:
cat_encd_train_final,cat_encd_test_final=cat_encd_train.align(cat_encd_test,join='inner',axis=1)
cat_encd_test_final.shape###aligning train & test data one hot encoded catg columns due to unqual no of columns i.e no of cilumns would differ for that we align to get same

(365, 200)

In [7]:
x_train_final=pd.concat([x_train[con_cols],cat_encd_train_final],axis=1)


In [8]:
x_test_final=pd.concat([x_test[con_cols],cat_encd_test_final],axis=1)

In [9]:
dtree_reg=DecisionTreeRegressor()

In [10]:
dtree_reg.fit(x_train_final,y_train)

DecisionTreeRegressor()

In [11]:
y_test_pred=dtree_reg.predict(x_test_final)
y_test_pred

array([107400., 126500., 165500., 236000.,  85000.,  79500., 187500.,
       110000., 318061., 159500., 128000.,  98600., 154000., 221000.,
       274970., 207500., 236000., 112500., 115000., 193000., 109500.,
       107000., 165000., 328000., 174000., 157900., 192000., 143000.,
       165600., 174000., 205950., 107500., 100000., 134500., 159434.,
       118500.,  88000., 162500., 138500., 129000., 168000.,  55000.,
       215000.,  91000., 310000., 198500., 426000., 104000., 194500.,
       124000., 156500., 207000., 248000., 205000., 144000., 285000.,
       378500., 138500., 202500., 144000.,  96500., 101800.,  80500.,
       164000., 339750., 154000., 214000., 311500., 130000.,  86000.,
       179000., 241500., 113000., 134450., 178000., 174900., 105000.,
       237000., 213250., 138500., 165400., 218000., 262500., 179000.,
       260400., 177000., 158500., 165600., 325300.,  68400., 133000.,
       186700., 126000., 140000., 113000., 179600.,  94000., 176000.,
       138500., 3480

In [12]:
dtree_reg.score(x_test_final,y_test)# R2 score for test data

0.7914889009625663

In [13]:
dtree_reg.score(x_train_final,y_train)

1.0

In [14]:
r2_score(y_test,y_test_pred)

0.7914889009625663